In [1]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import trimesh
import argparse
import os
import time
import numpy as np
import torch
import torch.optim as optim
import lib.utils as utils
from lib.utils import standard_normal_logprob
from lib.utils import count_nfe, count_total_time
from lib.utils import build_model_tabular
import pandas as pd
import random
from lib.visualize_flow import visualize_transform
import lib.layers.odefunc as odefunc
from matplotlib import pyplot as plt
%matplotlib inline 
import glob

In [2]:
SOLVERS = ["dopri5"]
parser = argparse.ArgumentParser('SoftFlow')
parser.add_argument(
    '--data', choices=['2spirals_1d','2spirals_2d', 'swissroll_1d','swissroll_2d', 'circles_1d', 'circles_2d', '2sines_1d', 'target_1d'],
    type=str, default='3d_microgel'
)
parser.add_argument("--layer_type", type=str, default="concatsquash", choices=["concatsquash"])
parser.add_argument('--dims', type=str, default='64-64-64')
parser.add_argument("--num_blocks", type=int, default=1, help='Number of stacked CNFs.')
parser.add_argument('--time_length', type=float, default=0.5)
parser.add_argument('--train_T', type=eval, default=True)
parser.add_argument("--divergence_fn", type=str, default="brute_force", choices=["brute_force", "approximate"])
parser.add_argument("--nonlinearity", type=str, default="tanh", choices=odefunc.NONLINEARITIES)

parser.add_argument('--solver', type=str, default='dopri5', choices=SOLVERS)
parser.add_argument('--atol', type=float, default=1e-5)
parser.add_argument('--rtol', type=float, default=1e-5)

parser.add_argument('--residual', type=eval, default=False, choices=[True, False])
parser.add_argument('--rademacher', type=eval, default=False, choices=[True, False])
parser.add_argument('--spectral_norm', type=eval, default=False, choices=[True, False])
parser.add_argument('--niters', type=int, default=100000)
parser.add_argument('--batch_size', type=int, default=100)
parser.add_argument('--test_batch_size', type=int, default=1000)
parser.add_argument('--lr', type=float, default=1e-3)
parser.add_argument('--weight_decay', type=float, default=1e-5)

# for the proposed method
parser.add_argument('--std_min', type=float, default=0.0)
parser.add_argument('--std_max', type=float, default=0.1)
parser.add_argument('--std_weight', type=float, default=2)

parser.add_argument('--viz_freq', type=int, default=100)
parser.add_argument('--val_freq', type=int, default=400)
parser.add_argument('--log_freq', type=int, default=10)
parser.add_argument('--gpu', type=int, default=0)
# args = parser.parse_args()
args, unknown = parser.parse_known_args()


In [3]:
# logger
args.data = "microgel_multiclass"
save_path = './results/' + args.data + '/SoftFlow'
utils.makedirs(save_path)

device = torch.device('cuda:' + str(args.gpu) if torch.cuda.is_available() else 'cpu')

In [4]:
def get_transforms(model):

    def sample_fn(z, logpz=None):
        zeors_std = torch.zeros(z.shape[0], 1).to(z)
        if logpz is not None:
            return model(z, zeors_std, logpz, reverse=True)
        else:
            return model(z, zeors_std, reverse=True)

    def density_fn(x, logpx=None):
        zeors_std = torch.zeros(x.shape[0], 1).to(x)
        if logpx is not None:
            return model(x, zeors_std, logpx, reverse=False)
        else:
            return model(x, zeors_std, reverse=False)

    return sample_fn, density_fn

DATA_add = "../../../multitemprature_data/NR_PAINT_TemperatureMeasurements/ViSP Results 21C/"
DATA_add = "../../../multitemprature_data/NR_PAINT_TemperatureMeasurements/"
temperature_list = os.listdir(DATA_add)
temperature_list.sort()
temperature_list= temperature_list[1:]


def get_data():
    idt = random.randint(0,len(temperature_list)-1)
    DATA_list = glob.glob(DATA_add+ temperature_list[idt]+"/*.3d")
    temperature = int(temperature_list[0][-3:-1])
    indx = random.randint(0,len(DATA_list)-1)
    DATA = pd.read_csv(DATA_list[indx], delimiter = "\t", header=None)
    DATA = DATA.to_numpy()[:,:3]
    DATA -= np.mean(DATA, axis=0) 
    DATA = (DATA-np.min(DATA, axis=0))/(np.max(DATA, axis=0)-np.min(DATA, axis=0)) 
    newrow = [temperature, temperature, temperature]
    DATA = np.vstack([newrow, DATA])

    return DATA



def compute_loss(args, model, batch_size=None):
    if batch_size is None: batch_size = args.batch_size
    x = get_data()


    x = torch.from_numpy(x).type(torch.float32).to(device)
    zero = torch.zeros(x.shape[0], 1).to(x)

    # transform to z
    std = (args.std_max - args.std_min) * torch.rand_like(x[:,0]).view(-1,1) + args.std_min
    eps = torch.randn_like(x) * std
    std_in = std / args.std_max * args.std_weight
#     print(x.shape, std_in.shape, zero.shape)
    z, delta_logp = model(x+eps, std_in, zero)

    # compute log q(z)
    logpz = standard_normal_logprob(z).sum(1, keepdim=True)

    logpx = logpz - delta_logp
    loss = -torch.mean(logpx)
    return loss

In [5]:
model = build_model_tabular(args, 3).to(device)

optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

time_meter = utils.RunningAverageMeter(0.93)
loss_meter = utils.RunningAverageMeter(0.93)
nfef_meter = utils.RunningAverageMeter(0.93)
nfeb_meter = utils.RunningAverageMeter(0.93)
tt_meter = utils.RunningAverageMeter(0.93)

end = time.time()
best_loss = float('inf')

In [6]:
LOW = -4
HIGH = 4
def visualize3d(samples, prior_sample, prior_logdensity, n_samples, transform, inverse_transform, memory=100, title1="$x ~ p(x)$", title2="$q(x)$", title3="$x ~ q(x)$", device="cpu", npts = 100):
    ## first one
    fig = plt.figure(figsize=(25,5))
    ax1 = fig.add_subplot(1, 3, 1, projection="3d")
    ax1.scatter(samples[:, 0], samples[:, 1], samples[:, 2], s=1, c='b', marker="s", facecolor="red", lw=0, alpha=1)
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_title(title1)
    
    ## second one
    side = np.linspace(LOW, HIGH, npts)
    xx, yy, zz = np.meshgrid(side, side, side)
    memory=100
    x = np.hstack([xx.reshape(-1, 1), yy.reshape(-1, 1), zz.reshape(-1, 1)])
    x = torch.from_numpy(x).type(torch.float32).to(device)
    zeros = torch.zeros(x.shape[0], 1).to(x)

    z, delta_logp = [], []
    inds = torch.arange(0, x.shape[0]).to(torch.int64)
    for ii in torch.split(inds, int(memory**2)):
        z_, delta_logp_ = inverse_transform(x[ii], zeros[ii])
        z.append(z_)
        delta_logp.append(delta_logp_)
    z = torch.cat(z, 0)
    delta_logp = torch.cat(delta_logp, 0)

    logpz = prior_logdensity(z).view(z.shape[0], -1).sum(1, keepdim=True)  # logp(z)
    logpx = logpz - delta_logp

    px = np.exp(logpx.detach().cpu().numpy()).reshape(npts, npts,npts)
    ax = fig.add_subplot(1, 3, 2)
    ax.imshow(px[:,:,1])
    ax.set_title(title2)
    
    ## Third one
    z = prior_sample(n_samples, 3).type(torch.float32).to(device)
    zk = []
    inds = torch.arange(0, z.shape[0]).to(torch.int64)
    for ii in torch.split(inds, int(memory**2)):
        zk.append(transform(z[ii]))

    zk = torch.cat(zk, 0).detach().cpu().numpy()

    ax1 = fig.add_subplot(1, 3, 3, projection="3d")
    ax1.scatter(zk[:, 0], zk[:, 1], zk[:, 2], s=1, c='r', marker="s", facecolor="red", lw=0, alpha=1)
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_title(title3)


In [ ]:
model.train()
for itr in range(1, args.niters + 1):
    optimizer.zero_grad()

    loss = compute_loss(args, model)
    

    loss_meter.update(loss.item())

    total_time = count_total_time(model)
    nfe_forward = count_nfe(model)

    loss.backward()
    optimizer.step()

    nfe_total = count_nfe(model)
    nfe_backward = nfe_total - nfe_forward
    nfef_meter.update(nfe_forward)
    nfeb_meter.update(nfe_backward)

    time_meter.update(time.time() - end)
    tt_meter.update(total_time)

    log_message = (
        'Iter {:04d} | Time {:.4f}({:.4f}) | Loss {:.6f}({:.6f}) | NFE Forward {:.0f}({:.1f})'
        ' | NFE Backward {:.0f}({:.1f}) | CNF Time {:.4f}({:.4f})'.format(
            itr, time_meter.val, time_meter.avg, loss_meter.val, loss_meter.avg, nfef_meter.val, nfef_meter.avg,
            nfeb_meter.val, nfeb_meter.avg, tt_meter.val, tt_meter.avg
        )
    )

    print(log_message)
    
    if itr % args.val_freq == 0 or itr == args.niters:
        with torch.no_grad():
            model.eval()
            test_loss = compute_loss(args, model, batch_size=args.test_batch_size)
            test_nfe = count_nfe(model)
            log_message = '[TEST] Iter {:04d} | Test Loss {:.6f} | NFE {:.0f}'.format(itr, test_loss, test_nfe)
            print(log_message)

            if test_loss.item() < best_loss:
                best_loss = test_loss.item()
                utils.makedirs(save_path)
                torch.save({
                    'args': args,
                    'state_dict': model.state_dict(),
                }, os.path.join(save_path, 'checkpt.pth'))
            model.train()

    if itr % args.viz_freq == 0:
        with torch.no_grad():
            model.eval()
#             p_samples = toy_data.inf_train_gen(args.data, batch_size=2000)
            p_samples = get_data()

            sample_fn, density_fn = get_transforms(model)
            p_samples = p_samples[1:]
            plt.figure(figsize=(9, 3))
            visualize3d(p_samples, torch.randn, standard_normal_logprob, p_samples.shape[0], sample_fn,density_fn, device=device)
            fig_filename = os.path.join(save_path, 'figs', '{:04d}.jpg'.format(itr))
            utils.makedirs(os.path.dirname(fig_filename))
            plt.savefig(fig_filename, format='png')#, dpi=1200)
            plt.close()
            model.train()

    end = time.time()

print('Training has finished.')

Iter 0001 | Time 0.7540(0.7540) | Loss 3.709614(3.709614) | NFE Forward 20(20.0) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0002 | Time 0.1823(0.7140) | Loss 3.269952(3.678838) | NFE Forward 20(20.0) | NFE Backward 20(25.6) | CNF Time 0.5000(0.5000)
Iter 0003 | Time 0.2092(0.6787) | Loss 3.689194(3.679563) | NFE Forward 20(20.0) | NFE Backward 26(25.6) | CNF Time 0.5000(0.5000)
Iter 0004 | Time 0.1835(0.6440) | Loss 3.184643(3.644918) | NFE Forward 20(20.0) | NFE Backward 20(25.2) | CNF Time 0.5000(0.5000)
Iter 0005 | Time 0.2129(0.6138) | Loss 3.167903(3.611527) | NFE Forward 20(20.0) | NFE Backward 26(25.3) | CNF Time 0.5000(0.5000)
Iter 0006 | Time 0.2086(0.5855) | Loss 3.848033(3.628083) | NFE Forward 20(20.0) | NFE Backward 26(25.3) | CNF Time 0.5000(0.5000)
Iter 0007 | Time 0.2122(0.5593) | Loss 3.098831(3.591035) | NFE Forward 20(20.0) | NFE Backward 20(24.9) | CNF Time 0.5000(0.5000)
Iter 0008 | Time 0.2087(0.5348) | Loss 3.627389(3.593580) | NFE Forward 20(20.0) | 

Iter 0064 | Time 0.2253(0.2348) | Loss 3.022399(3.308176) | NFE Forward 26(25.6) | NFE Backward 26(25.9) | CNF Time 0.5000(0.5000)
Iter 0065 | Time 0.2355(0.2348) | Loss 3.179669(3.299180) | NFE Forward 32(26.1) | NFE Backward 26(25.9) | CNF Time 0.5000(0.5000)
Iter 0066 | Time 0.2362(0.2349) | Loss 2.933122(3.273556) | NFE Forward 32(26.5) | NFE Backward 26(25.9) | CNF Time 0.5000(0.5000)
Iter 0067 | Time 0.2355(0.2350) | Loss 3.218143(3.269677) | NFE Forward 32(26.9) | NFE Backward 26(25.9) | CNF Time 0.5000(0.5000)
Iter 0068 | Time 0.2477(0.2358) | Loss 3.746896(3.303083) | NFE Forward 32(27.2) | NFE Backward 26(25.9) | CNF Time 0.5000(0.5000)
Iter 0069 | Time 0.2473(0.2366) | Loss 2.918787(3.276182) | NFE Forward 32(27.6) | NFE Backward 26(25.9) | CNF Time 0.5000(0.5000)
Iter 0070 | Time 0.2561(0.2380) | Loss 2.707664(3.236386) | NFE Forward 32(27.9) | NFE Backward 26(25.9) | CNF Time 0.5000(0.5000)
Iter 0071 | Time 0.2819(0.2411) | Loss 2.879283(3.211388) | NFE Forward 32(28.2) | 

Iter 0127 | Time 0.4398(0.3948) | Loss 0.833808(1.414510) | NFE Forward 50(46.5) | NFE Backward 50(45.3) | CNF Time 0.5000(0.5000)
Iter 0128 | Time 0.4245(0.3969) | Loss 1.533162(1.422816) | NFE Forward 50(46.7) | NFE Backward 50(45.7) | CNF Time 0.5000(0.5000)
Iter 0129 | Time 0.4076(0.3976) | Loss 1.176027(1.405541) | NFE Forward 44(46.5) | NFE Backward 50(46.0) | CNF Time 0.5000(0.5000)
Iter 0130 | Time 0.4238(0.3994) | Loss 1.187144(1.390253) | NFE Forward 50(46.8) | NFE Backward 50(46.2) | CNF Time 0.5000(0.5000)
Iter 0131 | Time 0.4032(0.3997) | Loss 1.922130(1.427484) | NFE Forward 44(46.6) | NFE Backward 50(46.5) | CNF Time 0.5000(0.5000)
Iter 0132 | Time 0.4140(0.4007) | Loss 0.844309(1.386662) | NFE Forward 50(46.8) | NFE Backward 44(46.3) | CNF Time 0.5000(0.5000)
Iter 0133 | Time 0.4284(0.4026) | Loss 0.818328(1.346879) | NFE Forward 50(47.0) | NFE Backward 50(46.6) | CNF Time 0.5000(0.5000)
Iter 0134 | Time 0.4070(0.4029) | Loss 1.718508(1.372893) | NFE Forward 44(46.8) | 

Iter 0190 | Time 0.4025(0.4161) | Loss 0.235368(0.299996) | NFE Forward 50(49.8) | NFE Backward 44(47.5) | CNF Time 0.5000(0.5000)
Iter 0191 | Time 0.4671(0.4196) | Loss -0.906213(0.215562) | NFE Forward 50(49.8) | NFE Backward 56(48.1) | CNF Time 0.5000(0.5000)
Iter 0192 | Time 0.4352(0.4207) | Loss 0.627489(0.244397) | NFE Forward 50(49.8) | NFE Backward 56(48.7) | CNF Time 0.5000(0.5000)
Iter 0193 | Time 0.4442(0.4224) | Loss -0.458460(0.195197) | NFE Forward 50(49.8) | NFE Backward 56(49.2) | CNF Time 0.5000(0.5000)
Iter 0194 | Time 0.3793(0.4194) | Loss 0.050006(0.185033) | NFE Forward 50(49.8) | NFE Backward 44(48.8) | CNF Time 0.5000(0.5000)
Iter 0195 | Time 0.3874(0.4171) | Loss -0.047559(0.168752) | NFE Forward 50(49.9) | NFE Backward 44(48.5) | CNF Time 0.5000(0.5000)
Iter 0196 | Time 0.4174(0.4171) | Loss -1.091753(0.080517) | NFE Forward 50(49.9) | NFE Backward 50(48.6) | CNF Time 0.5000(0.5000)
Iter 0197 | Time 0.4388(0.4187) | Loss -1.159245(-0.006267) | NFE Forward 50(49

Iter 0253 | Time 0.4037(0.4113) | Loss 0.295917(0.229860) | NFE Forward 50(50.0) | NFE Backward 44(46.1) | CNF Time 0.5000(0.5000)
Iter 0254 | Time 0.4061(0.4109) | Loss 0.214375(0.228776) | NFE Forward 50(50.0) | NFE Backward 44(46.0) | CNF Time 0.5000(0.5000)
Iter 0255 | Time 0.3978(0.4100) | Loss 0.271083(0.231737) | NFE Forward 50(50.0) | NFE Backward 44(45.9) | CNF Time 0.5000(0.5000)
Iter 0256 | Time 0.3967(0.4090) | Loss -0.131089(0.206339) | NFE Forward 50(50.0) | NFE Backward 44(45.7) | CNF Time 0.5000(0.5000)
Iter 0257 | Time 0.4547(0.4122) | Loss 0.530421(0.229025) | NFE Forward 50(50.0) | NFE Backward 56(46.4) | CNF Time 0.5000(0.5000)
Iter 0258 | Time 0.3986(0.4113) | Loss -0.149757(0.202510) | NFE Forward 50(50.0) | NFE Backward 44(46.3) | CNF Time 0.5000(0.5000)
Iter 0259 | Time 0.4609(0.4148) | Loss -0.111983(0.180496) | NFE Forward 50(50.0) | NFE Backward 56(47.0) | CNF Time 0.5000(0.5000)
Iter 0260 | Time 0.3987(0.4136) | Loss 0.209599(0.182533) | NFE Forward 50(50.0)

...

Iter 26921 | Time 0.6427(0.6510) | Loss -0.708296(-0.752941) | NFE Forward 80(77.4) | NFE Backward 68(70.3) | CNF Time 0.5000(0.5000)
Iter 26922 | Time 0.6145(0.6484) | Loss -0.806177(-0.756668) | NFE Forward 68(76.7) | NFE Backward 68(70.1) | CNF Time 0.5000(0.5000)
Iter 26923 | Time 0.5688(0.6428) | Loss 0.191163(-0.690320) | NFE Forward 80(77.0) | NFE Backward 68(70.0) | CNF Time 0.5000(0.5000)
Iter 26924 | Time 0.6049(0.6402) | Loss -0.558785(-0.681112) | NFE Forward 74(76.7) | NFE Backward 68(69.9) | CNF Time 0.5000(0.5000)
Iter 26925 | Time 0.6403(0.6402) | Loss -0.995296(-0.703105) | NFE Forward 86(77.4) | NFE Backward 68(69.7) | CNF Time 0.5000(0.5000)
Iter 26926 | Time 0.6145(0.6384) | Loss -1.001636(-0.724002) | NFE Forward 74(77.2) | NFE Backward 68(69.6) | CNF Time 0.5000(0.5000)
Iter 26927 | Time 0.6044(0.6360) | Loss -0.606800(-0.715798) | NFE Forward 68(76.5) | NFE Backward 68(69.5) | CNF Time 0.5000(0.5000)
Iter 26928 | Time 0.7460(0.6437) | Loss -0.858654(-0.725798) | 

Iter 26983 | Time 0.7101(0.6620) | Loss -0.565862(-0.623245) | NFE Forward 86(78.7) | NFE Backward 74(71.1) | CNF Time 0.5000(0.5000)
Iter 26984 | Time 0.6551(0.6616) | Loss -0.560391(-0.618845) | NFE Forward 86(79.2) | NFE Backward 68(70.9) | CNF Time 0.5000(0.5000)
Iter 26985 | Time 0.7010(0.6643) | Loss -0.489199(-0.609770) | NFE Forward 80(79.3) | NFE Backward 80(71.5) | CNF Time 0.5000(0.5000)
Iter 26986 | Time 0.6425(0.6628) | Loss -0.528530(-0.604083) | NFE Forward 68(78.5) | NFE Backward 74(71.7) | CNF Time 0.5000(0.5000)
Iter 26987 | Time 0.6803(0.6640) | Loss -1.327954(-0.654754) | NFE Forward 74(78.2) | NFE Backward 68(71.5) | CNF Time 0.5000(0.5000)
Iter 26988 | Time 0.6523(0.6632) | Loss -0.904613(-0.672244) | NFE Forward 74(77.9) | NFE Backward 68(71.2) | CNF Time 0.5000(0.5000)
Iter 26989 | Time 0.6143(0.6598) | Loss -0.646849(-0.670467) | NFE Forward 68(77.2) | NFE Backward 68(71.0) | CNF Time 0.5000(0.5000)
Iter 26990 | Time 0.6822(0.6613) | Loss -0.539521(-0.661300) |

Iter 27045 | Time 0.6241(0.6594) | Loss -0.235294(-0.794626) | NFE Forward 86(77.6) | NFE Backward 68(70.8) | CNF Time 0.5000(0.5000)
Iter 27046 | Time 0.6282(0.6572) | Loss -1.478044(-0.842465) | NFE Forward 74(77.3) | NFE Backward 68(70.6) | CNF Time 0.5000(0.5000)
Iter 27047 | Time 0.5741(0.6514) | Loss -1.642259(-0.898450) | NFE Forward 56(75.8) | NFE Backward 68(70.4) | CNF Time 0.5000(0.5000)
Iter 27048 | Time 0.6414(0.6507) | Loss -0.882731(-0.897350) | NFE Forward 80(76.1) | NFE Backward 68(70.2) | CNF Time 0.5000(0.5000)
Iter 27049 | Time 0.6653(0.6517) | Loss -0.852010(-0.894176) | NFE Forward 74(76.0) | NFE Backward 68(70.1) | CNF Time 0.5000(0.5000)
Iter 27050 | Time 0.6915(0.6545) | Loss 0.087618(-0.825451) | NFE Forward 62(75.0) | NFE Backward 80(70.8) | CNF Time 0.5000(0.5000)
Iter 27051 | Time 0.7551(0.6615) | Loss -0.292974(-0.788177) | NFE Forward 92(76.2) | NFE Backward 80(71.4) | CNF Time 0.5000(0.5000)
Iter 27052 | Time 0.7183(0.6655) | Loss -0.069365(-0.737860) | 

Iter 27107 | Time 0.6162(0.6685) | Loss -0.294929(-0.778108) | NFE Forward 74(77.8) | NFE Backward 68(72.3) | CNF Time 0.5000(0.5000)
Iter 27108 | Time 0.6528(0.6674) | Loss -0.249690(-0.741119) | NFE Forward 80(77.9) | NFE Backward 68(72.0) | CNF Time 0.5000(0.5000)
Iter 27109 | Time 0.6202(0.6641) | Loss -0.435976(-0.719759) | NFE Forward 86(78.5) | NFE Backward 68(71.7) | CNF Time 0.5000(0.5000)
Iter 27110 | Time 0.6301(0.6617) | Loss -1.597327(-0.781189) | NFE Forward 74(78.2) | NFE Backward 68(71.4) | CNF Time 0.5000(0.5000)
Iter 27111 | Time 0.6168(0.6586) | Loss -0.773214(-0.780630) | NFE Forward 74(77.9) | NFE Backward 68(71.2) | CNF Time 0.5000(0.5000)
Iter 27112 | Time 0.6367(0.6571) | Loss -1.148446(-0.806377) | NFE Forward 80(78.0) | NFE Backward 68(71.0) | CNF Time 0.5000(0.5000)
Iter 27113 | Time 0.5742(0.6513) | Loss -0.503703(-0.785190) | NFE Forward 56(76.5) | NFE Backward 68(70.8) | CNF Time 0.5000(0.5000)
Iter 27114 | Time 0.6638(0.6521) | Loss -1.393457(-0.827769) |

Iter 27169 | Time 0.7418(0.6616) | Loss -0.648656(-0.771514) | NFE Forward 92(77.9) | NFE Backward 80(72.5) | CNF Time 0.5000(0.5000)
Iter 27170 | Time 0.7236(0.6659) | Loss -0.529471(-0.754571) | NFE Forward 92(78.9) | NFE Backward 80(73.0) | CNF Time 0.5000(0.5000)
Iter 27171 | Time 0.6946(0.6679) | Loss -0.988907(-0.770975) | NFE Forward 74(78.6) | NFE Backward 80(73.5) | CNF Time 0.5000(0.5000)
Iter 27172 | Time 0.6390(0.6659) | Loss -0.177774(-0.729451) | NFE Forward 74(78.3) | NFE Backward 68(73.1) | CNF Time 0.5000(0.5000)
Iter 27173 | Time 0.6883(0.6674) | Loss -0.485964(-0.712407) | NFE Forward 80(78.4) | NFE Backward 74(73.2) | CNF Time 0.5000(0.5000)
Iter 27174 | Time 0.6315(0.6649) | Loss -1.154993(-0.743388) | NFE Forward 68(77.7) | NFE Backward 68(72.8) | CNF Time 0.5000(0.5000)
Iter 27175 | Time 0.6364(0.6629) | Loss -1.177100(-0.773748) | NFE Forward 74(77.4) | NFE Backward 68(72.5) | CNF Time 0.5000(0.5000)
Iter 27176 | Time 0.6771(0.6639) | Loss -1.423255(-0.819213) |